#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data 


In [2]:
import pymysql
import sqlalchemy as db
import getpass

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/search?tableCount%5B%5D=0-10&tableCount%5B%5D=10-30&dataType%5B%5D=Numeric&databaseSize%5B%5D=KB&databaseSize%5B%5D=MB)

In [3]:
password = getpass.getpass("Insert your mysql root password: ")
engine = db.create_engine('mysql+pymysql://guest:{}@relational.fit.cvut.cz/stats'.format(password))
print("Connected to server!")

Insert your mysql root password: ········
Connected to server!


#### 4. Import the users table 

In [4]:
query = "SELECT * FROM users;"

users = pd.read_sql_query(query, engine)
display(users.head())

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None


#### 5. Rename Id column to userId

In [5]:
users = users.rename(columns = {"Id":"userId"})
display(users.head())

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None


#### 6. Import the posts table. 

In [6]:
query = "SELECT * FROM posts;"

posts = pd.read_sql_query(query, engine)
display(posts.head())

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaT,NaT,NaN,NaT,None,None
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaT,NaN,NaT,None,None
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaT,None,None
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaT,NaT,NaN,NaT,None,None
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,None,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaT,None,None


#### 7. Rename Id column to postId and OwnerUserId to userId

In [42]:
posts = posts.rename(columns = {"Id":"postId","OwnerUserId":"userId"})
display(posts.head())

,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [43]:
users = users[["userId","Reputation","Views","UpVotes","DownVotes"]]
posts = posts[["postId","Score","userId","ViewCount","CommentCount"]]

display(users.head())
display(posts.head())

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5


,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [9]:
merge_table = users.merge(posts,on="userId")
display(merge_table.head())

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0


#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [10]:
null_cols = merge_table.isnull().sum()
null_cols[null_cols > 0]

ViewCount    48396
dtype: int64

In [11]:
total_rows = merge_table["ViewCount"].shape[0]
empty_values = merge_table["ViewCount"].isnull().sum()
not_empty_values = sum(list(merge_table["ViewCount"].value_counts().values))
print(total_rows, empty_values+not_empty_values)
print(empty_values,not_empty_values)

90584 90584
48396 42188


#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [12]:
print("Al ser 50/50 es imposible saber que hacer con los datos sin saber el objetivo del analisis")
print("Si fuese necesario para el analisis saber el ViewCount se podría utilizar una muestra significativa que contenga dicho campo y extrapolar los resultados")

Al ser 50/50 es imposible saber que hacer con los datos sin saber el objetivo del analisis
Si fuese necesario para el analisis saber el ViewCount se podría utilizar una muestra significativa que contenga dicho campo y extrapolar los resultados


#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [13]:
print(merge_table.dtypes)
print("\n A changed is made to ViewCount, the column with the issue\n")
print(merge_table.astype({'ViewCount': 'object'}).dtypes)

userId            int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
postId            int64
Score             int64
ViewCount       float64
CommentCount      int64
dtype: object

 A changed is made to ViewCount, the column with the issue

userId           int64
Reputation       int64
Views            int64
UpVotes          int64
DownVotes        int64
postId           int64
Score            int64
ViewCount       object
CommentCount     int64
dtype: object


#### Bonus: Identify extreme values in your merged dataframe as you have learned in class, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder.

In [62]:
stats = merge_table.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

SyntaxError: invalid syntax (<ipython-input-62-08a07e18d4c9>, line 1)

In [76]:
outliers = pd.DataFrame(columns=merge_table.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 3
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = merge_table[(merge_table[col] < lower) | 
                   (merge_table[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)

In [77]:
display(outliers)

,CommentCount,DownVotes,Outlier,Reputation,Score,UpVotes,ViewCount,Views,postId,userId
1733,4,59,Reputation,18283,10,1014,NaN,3781,167,159
1734,0,59,Reputation,18283,2,1014,NaN,3781,169,159
1735,0,59,Reputation,18283,8,1014,NaN,3781,171,159
1736,0,59,Reputation,18283,35,1014,NaN,3781,174,159
1737,2,59,Reputation,18283,22,1014,NaN,3781,177,159
1738,0,59,Reputation,18283,12,1014,NaN,3781,179,159
1739,0,59,Reputation,18283,50,1014,39097.0,3781,181,159
1740,0,59,Reputation,18283,6,1014,NaN,3781,184,159
1741,3,59,Reputation,18283,51,1014,9772.0,3781,213,159
1742,2,59,Reputation,18283,15,1014,NaN,3781,221,159


In [69]:
outliers.to_csv(r'../laba-data_cleaning\outliers.csv')